# Assignment 4 - Data Science in Finance and Insurance
Submission by: Dennis Goldenberg

In [6]:
#Import Statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

## Read in Data, Scale Distances

In [54]:
K = 7;
#read in data
rawData = pd.read_csv("data.csv")

#find the 7 nearest neighbors to each, use p = 1 for l1-norm
nn = NearestNeighbors(n_neighbors= K, p = 1).fit(rawData);

#Get distances
distances = nn.kneighbors()[0];
#find max distances - using the 7th nearest neighbor as they are naturally ordered
maxdistances = np.expand_dims(nn.kneighbors()[0][:,6], axis = 1)

scaled_dist = distances/maxdistances

## Derive Weights using tri-cube function